# PYTORCH 

#### Net 설정

1. Network 는 nn.Module을 상속받아야 한다. 
2. nn.Conv2d(A,B,C)  
 -  A:input image Channel 
 -  B: output Channel 
 -  C: Size CxC Square convolution
 3. convolution 2개를 연속으로 붙일 때는  size[ PRE output] == size[ AFTER input ] 
 4. network를 커널이라고 부르기도 함.
 5. Relu 는 torch.nn.fuctional 에 있음. 주로 F로 aliasing.
 6.  tensor.view()를 통해서 dimension을 바꿀 수 있음. 

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

sample_number = 64
channel = 3
height = 32
width = 32

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        
        self.conv1 = nn.Conv2d(channel, 6, 3)  
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 3, 3, 3])


In [7]:
input = torch.randn(sample_number, channel, height, width)
out = net(input)

In [8]:
net.zero_grad()
out.backward(torch.randn(sample_number, 10))

In [9]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

C:\Users\beomj\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:443: UserWarning: Using a target size (torch.Size([1, 10])) that is different to the input size (torch.Size([64, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(1.7805, grad_fn=<MseLossBackward>)


In [10]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [11]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0171, -0.0030,  0.0185,  0.0365, -0.0053, -0.0153])


In [12]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [20]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
for i in range(500):
    output = net(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()    # Does the update
    if i%50 ==0:
        print(f"EPOCH {i}", loss)
print(f"finished: {loss}")
    

EPOCH 0 tensor(3.0724e-06, grad_fn=<MseLossBackward>)
EPOCH 50 tensor(1.9739e-06, grad_fn=<MseLossBackward>)
EPOCH 100 tensor(1.0682e-06, grad_fn=<MseLossBackward>)
EPOCH 150 tensor(2.0152e-06, grad_fn=<MseLossBackward>)
EPOCH 200 tensor(2.4571e-06, grad_fn=<MseLossBackward>)
EPOCH 250 tensor(1.3690e-06, grad_fn=<MseLossBackward>)
EPOCH 300 tensor(4.8449e-07, grad_fn=<MseLossBackward>)
EPOCH 350 tensor(1.4407e-06, grad_fn=<MseLossBackward>)
EPOCH 400 tensor(1.9935e-06, grad_fn=<MseLossBackward>)
EPOCH 450 tensor(5.3141e-07, grad_fn=<MseLossBackward>)
finished: 3.0090683367234305e-07
